In [15]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

# Helper function to add ordinal suffix to numbers (1st, 2nd, 3rd, etc.)
def ordinal_suffix(n):
    if pd.isna(n):
        return ""
    n = int(n)
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return f"{n}{suffix}"

# Read input files
print("Reading input files...")
keys_df = pd.read_csv('KEYS.csv')
census_pop_df = pd.read_csv('CENSUS_POPULATION_STATE.csv')
census_mhi_df = pd.read_csv('CENSUS_MHI_STATE.csv')
redfin_df = pd.read_csv('REDFIN_MEDIAN_SALE_PRICE.csv')

# 1. Filter states from KEYS file (Column A formula)
# =QUERY(KEYS!A:G,"SELECT A WHERE (F = ""state"") AND NOT A LIKE ""%'%""")
print("Filtering states...")
states_df = keys_df[(keys_df['region_type'] == 'state') & (~keys_df['key_row'].str.contains("'"))]

# 2. Create output dataframe with key_row column
output_df = pd.DataFrame()
output_df['key_row'] = states_df['key_row'].sort_values().reset_index(drop=True)

# 3. Extract population data from census file (Column B formula)
# =INDEX(CENSUS_POPULATION_STATE!$A:$ZZ,MATCH("    Total population",CENSUS_POPULATION_STATE!$A:$A,0),
#        MATCH(INDEX(KEYS!$A:$G,MATCH(A2,KEYS!$A:$A,0),3)&"!!Estimate",CENSUS_POPULATION_STATE!$1:$1,0))
print("Extracting population data...")
population_data = []

for state_key in output_df['key_row']:
    # Get zillow_region_name for this state from KEYS
    zillow_name = keys_df.loc[keys_df['key_row'] == state_key, 'zillow_region_name'].values[0]
    
    # Find the column with this state's population estimate
    col_name = next((col for col in census_pop_df.columns if col.startswith(f"{zillow_name}!!Estimate")), None)
    
    # Find the row with "Total population"
    pop_rows = census_pop_df[census_pop_df['Label (Grouping)'].str.contains("Total population", case=False, na=False)]
    
    if not pop_rows.empty and col_name:
        pop_value = pop_rows.iloc[0][col_name]
        # Format with commas
        try:
            population = "{:,}".format(int(float(pop_value)))
        except:
            population = str(pop_value)
    else:
        population = ""
    
    population_data.append(population)

output_df['census_population'] = population_data

# 4. Create population ranking (Column C formula)
# =RANK(B2,$B$2:$B$53)&MID("thstndrdth",MIN(9,2*RIGHT(RANK(B2,$B$2:$B$53))*(MOD(RANK(B2,$B$2:$B$53)-11,100)>2)+1),2)
print("Creating population rankings...")
# Convert to numeric for ranking
output_df['census_population_numeric'] = pd.to_numeric(output_df['census_population'].str.replace(',', ''), errors='coerce')
population_ranks = output_df['census_population_numeric'].rank(method='min', ascending=False)
output_df['population_rank'] = population_ranks.apply(ordinal_suffix)

# 5. Create population blurbs (Column D formula)
# =INDEX(KEYS!$A:$H,MATCH(A2,KEYS!$A:$A,0),8)&" is "&C2&" in the nation in population among states, DC, and Puerto Rico."
print("Creating population blurbs...")
output_df['population_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} is {row['population_rank']} in the nation in population among states, DC, and Puerto Rico.",
    axis=1
)

# 6. Extract median household income (Column E formula)
# =INDEX(CENSUS_MHI_STATE!$A:$ZZ,MATCH("    Households",CENSUS_MHI_STATE!$A:$A,0),
#        MATCH(INDEX(KEYS!$A:$G,MATCH(A2,KEYS!$A:$A,0),3)&"!!Median income (dollars)!!Estimate",CENSUS_MHI_STATE!$1:$1,0))
print("Extracting median household income...")
income_data = []

for state_key in output_df['key_row']:
    # Get zillow_region_name for this state
    zillow_name = keys_df.loc[keys_df['key_row'] == state_key, 'zillow_region_name'].values[0]
    
    # Find the column with this state's median income estimate
    col_name = next((col for col in census_mhi_df.columns if col.startswith(f"{zillow_name}!!Median income (dollars)!!Estimate")), None)
    
    # Find the row with "Households"
    income_rows = census_mhi_df[census_mhi_df['Label (Grouping)'].str.contains("Households", case=False, na=False)]
    
    if not income_rows.empty and col_name:
        income_value = income_rows.iloc[0][col_name]
        # Format as currency
        try:
            income = "${:,}".format(int(float(income_value)))
        except:
            income = f"${income_value}" if income_value else ""
    else:
        income = ""
    
    income_data.append(income)

output_df['median_household_income'] = income_data
output_df['median_household_income_numeric'] = pd.to_numeric(output_df['median_household_income'].str.replace('[$,]', '', regex=True), errors='coerce')

# 7. Create income ranking (Column F formula)
# =RANK(E2,$E$2:$E$53)&MID("thstndrdth",MIN(9,2*RIGHT(RANK(E2,$E$2:$E$53))*(MOD(RANK(E2,$E$2:$E$53)-11,100)>2)+1),2)
print("Creating income rankings...")
income_ranks = output_df['median_household_income_numeric'].rank(method='min', ascending=False)
output_df['median_household_income_rank'] = income_ranks.apply(ordinal_suffix)

# 8. Create income blurbs (Column G formula)
# =INDEX(KEYS!$A:$H,MATCH(A2,KEYS!$A:$A,0),8)&" is "&IF(F2="1st","the highest",F2)&" in the nation in median household income among states, DC, and Puerto Rico."
print("Creating income blurbs...")
output_df['median_household_income_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} is {('the highest' if row['median_household_income_rank'] == '1st' else row['median_household_income_rank'])} in the nation in median household income among states, DC, and Puerto Rico.",
    axis=1
)

# 9. Extract median sale price (Column H formula)
# =IFERROR(VALUE(SUBSTITUTE(SUBSTITUTE(INDEX(REDFIN_MEDIAN_SALE_PRICE!$A$3:$ZZZ,
#          MATCH(INDEX(KEYS!$A:$G,MATCH(A2,KEYS!$A:$A,0),3),REDFIN_MEDIAN_SALE_PRICE!$A$3:$A,0),
#          MATCH(INDEX(REDFIN_MEDIAN_SALE_PRICE!$2:$2,1,COUNTA(REDFIN_MEDIAN_SALE_PRICE!$2:$2)),
#          REDFIN_MEDIAN_SALE_PRICE!$2:$2,0)),"K","000"),"$","")),"")
print("Extracting median sale prices...")
sale_prices = []
sale_prices_numeric = []

for state_key in output_df['key_row']:
    # Get zillow_region_name for this state
    zillow_name = keys_df.loc[keys_df['key_row'] == state_key, 'zillow_region_name'].values[0]
    
    try:
        # Find the row in redfin_df that corresponds to this state
        redfin_row = redfin_df[redfin_df.iloc[:, 0] == zillow_name]
        
        if not redfin_row.empty:
            # Get the last column value (most recent month)
            price_str = redfin_row.iloc[0, -1]
            
            # Clean the price string
            if isinstance(price_str, str):
                price_str = price_str.replace('$', '').replace('K', '000')
                price_numeric = float(price_str)
                price_formatted = "${:,}".format(int(price_numeric))
            else:
                price_numeric = float(price_str) if not pd.isna(price_str) else np.nan
                price_formatted = "${:,}".format(int(price_numeric)) if not pd.isna(price_numeric) else ""
        else:
            price_formatted = ""
            price_numeric = np.nan
    except Exception as e:
        print(f"Error processing sale price for {state_key}: {e}")
        price_formatted = ""
        price_numeric = np.nan
    
    sale_prices.append(price_formatted)
    sale_prices_numeric.append(price_numeric)

output_df['median_sale_price'] = sale_prices
output_df['median_sale_price_numeric'] = sale_prices_numeric

# 10. Create price ranking (Column I formula)
# =IFNA(RANK(H2,$H$2:$H$53)&MID("thstndrdth",MIN(9,2*RIGHT(RANK(H2,$H$2:$H$53))*(MOD(RANK(H2,$H$2:$H$53)-11,100)>2)+1),2),"")
print("Creating price rankings...")
price_ranks = output_df['median_sale_price_numeric'].rank(method='min', ascending=False)
output_df['median_sale_price_rank'] = price_ranks.apply(ordinal_suffix)

# 11. Get the date from the last column of redfin_df for the blurbs
last_column = redfin_df.columns[-1]
try:
    # Try to extract date using various methods
    date_match = re.search(r'(\w+ \d{4})', last_column)
    if date_match:
        date_str = date_match.group(1)
    else:
        # Default to February 2025 as shown in the output examples
        date_str = "February 2025"
except:
    date_str = "February 2025"

# 12. Create price blurbs (Column J formula)
# =INDEX(KEYS!$A:$H,MATCH(A2,KEYS!$A:$A,0),8)&" has the "&IF(I2="1st","single",I2)&" highest median sale price on homes in the nation among states, DC, and Puerto Rico, according to Redfin data from "&TEXT(INDEX(REDFIN_MEDIAN_SALE_PRICE!$2:$2,1,COUNTA(REDFIN_MEDIAN_SALE_PRICE!$2:$2)),"MMMM YYYY")&"."
print("Creating price blurbs...")
output_df['median_sale_price_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} has the {('single' if row['median_sale_price_rank'] == '1st' else row['median_sale_price_rank'])} highest median sale price on homes in the nation among states, DC, and Puerto Rico, according to Redfin data from {date_str}." if row['median_sale_price_rank'] else "",
    axis=1
)

# 13. Calculate house affordability ratio (Column K formula)
# =ROUND(H2/E2,1)
print("Calculating house affordability ratios...")
output_df['house_affordability_ratio'] = round(output_df['median_sale_price_numeric'] / output_df['median_household_income_numeric'], 1)

# 14. Create affordability ranking (Column L formula)
# =IFNA(RANK(K2,$K$2:$K$53,1)&MID("thstndrdth",MIN(9,2*RIGHT(RANK(K2,$K$2:$K$53,1))*(MOD(RANK(K2,$K$2:$K$53,1)-11,100)>2)+1),2),"")
print("Creating affordability rankings...")
# Note: ascending=True for affordability ratio because lower is better
affordability_ranks = output_df['house_affordability_ratio'].rank(method='min', ascending=True)
output_df['house_affordability_ratio_rank'] = affordability_ranks.apply(ordinal_suffix)

# 15. Create affordability blurbs (Column M formula)
# =INDEX(KEYS!$A:$H,MATCH(A2,KEYS!$A:$A,0),8)&" has the "&IF(L2="1st","single",L2)&" lowest house affordability ratio in the nation among states, DC, and Puerto Rico, according to Redfin data from "&TEXT(INDEX(REDFIN_MEDIAN_SALE_PRICE!$2:$2,1,COUNTA(REDFIN_MEDIAN_SALE_PRICE!$2:$2)),"MMMM YYYY")&"."
print("Creating affordability blurbs...")
output_df['house_affordability_ratio_blurb'] = output_df.apply(
    lambda row: f"{keys_df.loc[keys_df['key_row'] == row['key_row'], 'alternative_name'].values[0]} has the {('single' if row['house_affordability_ratio_rank'] == '1st' else row['house_affordability_ratio_rank'])} lowest house affordability ratio in the nation among states, DC, and Puerto Rico, according to Redfin data from {date_str}." if row['house_affordability_ratio_rank'] else "",
    axis=1
)

# Remove the _numeric columns used for calculations
output_df = output_df.drop(['census_population_numeric', 'median_household_income_numeric', 'median_sale_price_numeric'], axis=1)

# Save to CSV
print("Saving output to CSV...")
output_df.to_csv('output_formula_implementation.csv', index=False)

print("Process completed successfully!")

Reading input files...

Inspecting Census Population file:
['SEX AND AGE', '\xa0\xa0\xa0\xa0Total population', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Male', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Female', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Sex ratio (males per 100 females)', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Under 5 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa05 to 9 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa010 to 14 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa015 to 19 years', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa020 to 24 years']

Inspecting Census MHI file:
['HOUSEHOLD INCOME BY RACE AND HISPANIC OR LATINO ORIGIN OF HOUSEHOLDER', '\xa0\xa0\xa0\xa0Households', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0One race--', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0White', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Black or African American', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0American Indian and Alaska Native', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0Asian', '\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa